<a href="https://colab.research.google.com/github/Chaithanyaistharla/Chaithanya_INFO5731_Spring2025/blob/main/Istharla_Chaithanya_Assignment_02" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment 2**

In this assignment, you will work on gathering text data from an open data source via web scraping or API. Following this, you will need to clean the text data and perform syntactic analysis on the data. Follow the instructions carefully and design well-structured Python programs to address each question.

**Expectations**:
*   Use the provided .*ipynb* document to write your code & respond to the questions. Avoid generating a new file.
*   Write complete answers and run all the cells before submission.
*   Make sure the submission is "clean"; *i.e.*, no unnecessary code cells.
*   Once finished, allow shared rights from top right corner (*see Canvas for details*).

* **Make sure to submit the cleaned data CSV in the comment section - 10 points**

**Total points**: 100

**Deadline**: Monday, at 11:59 PM.

**Late Submission will have a penalty of 10% reduction for each day after the deadline.**

**Please check that the link you submitted can be opened and points to the correct assignment.**


# Question 1 (25 points)

Write a python program to collect text data from **either of the following sources** and save the data into a **csv file:**

(1) Collect all the customer reviews of a product (you can choose any porduct) on amazon. [atleast 1000 reviews]

(2) Collect the top 1000 User Reviews of a movie recently in 2023 or 2024 (you can choose any movie) from IMDB. [If one movie doesn't have sufficient reviews, collect reviews of atleast 2 or 3 movies]


(3) Collect the **abstracts** of the top 10000 research papers by using the query "machine learning", "data science", "artifical intelligence", or "information extraction" from Semantic Scholar.

(4) Collect all the information of the 904 narrators in the Densho Digital Repository.

(5)**Collect a total of 10000 reviews** of the top 100 most popular software from G2 and Capterra.


In [ ]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import time
from itertools import zip_longest

base_url = "https://www.amazon.com/Apple-iPhone-15-Pro-Titanium/dp/B0CMZG4M6H/ref=cm_cr_srp_mb_bdcrb_top?ie=UTF8&th=1"

cust_name, review_title, rate, review_content = [], [], [], []
total_reviews = 1000
reviews_per_page = 10
pages_to_scrape = total_reviews // reviews_per_page

for page in range(1, pages_to_scrape + 1):
    print(f"Scraping page {page}...")

    page_url = base_url.format(page, page)
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    response = requests.get(page_url, headers=headers)

    if response.status_code != 200:
        print(f"Failed to fetch page {page}, status code: {response.status_code}")
        break

    soup = bs(response.content, 'html.parser')

    names = [name.get_text().strip() for name in soup.find_all('span', class_='a-profile-name')]
    titles = [title.get_text(strip=True).replace("5.0 out of 5 stars\n", "") for title in soup.find_all('a', class_='review-title')]
    ratings = [rating.get_text().strip() for rating in soup.find_all('i', class_='review-rating')]
    reviews = [review.get_text(strip=True) for review in soup.find_all('span', {"data-hook": "review-body"})]

    # Ensure lists have equal length
    min_length = min(len(names), len(titles), len(ratings), len(reviews))

    for i in range(min_length):
        if names[i]:  # Ensures name is not empty
            cust_name.append(names[i])
            review_title.append(titles[i])
            rate.append(ratings[i])
            review_content.append(reviews[i])

    time.sleep(2)

# Creating DataFrame and saving to CSV
data = list(zip_longest(cust_name, review_title, rate, review_content, fillvalue="NULL"))
df = pd.DataFrame(data, columns=["Customer Name", "Review Title", "Rating", "Reviews"])
df.to_csv("amazon_reviews.csv", index=False)

print("Scraping completed! Data saved to amazon_reviews.csv")

Scraping page 1...
Scraping page 2...
Scraping page 3...
Scraping page 4...
Scraping page 5...
Scraping page 6...
Scraping page 7...
Scraping page 8...
Scraping page 9...
Scraping page 10...
Scraping page 11...
Scraping page 12...
Scraping page 13...
Scraping page 14...
Scraping page 15...
Scraping page 16...
Scraping page 17...
Scraping page 18...
Scraping page 19...
Scraping page 20...
Scraping page 21...
Scraping page 22...
Scraping page 23...
Scraping page 24...
Scraping page 25...
Scraping page 26...
Scraping page 27...
Scraping page 28...
Scraping page 29...
Scraping page 30...
Scraping page 31...
Scraping page 32...
Scraping page 33...
Scraping page 34...
Scraping page 35...
Scraping page 36...
Scraping page 37...
Scraping page 38...
Scraping page 39...
Scraping page 40...
Scraping page 41...
Scraping page 42...
Scraping page 43...
Scraping page 44...
Scraping page 45...
Scraping page 46...
Scraping page 47...
Scraping page 48...
Scraping page 49...
Scraping page 50...
Scraping 

# Question 2 (15 points)

Write a python program to **clean the text data** you collected in the previous question and save the clean data in a new column in the csv file. The data cleaning steps include: [Code and output is required for each part]

(1) Remove noise, such as special characters and punctuations.

(2) Remove numbers.

(3) Remove stopwords by using the stopwords list.

(4) Lowercase all texts

(5) Stemming.

(6) Lemmatization.

In [2]:
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# Download necessary NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')

# Load NLTK stopwords
stop_words = set(stopwords.words('english'))

# Load the dataset
try:
    df = pd.read_csv("amazon_reviews.csv")
    print("Dataset loaded successfully!")
except FileNotFoundError:
    print("Error: amazon_reviews.csv not found!")
    exit()

# Display the first few rows
print(df.head())

# Text cleaning function: remove special characters, numbers, and stopwords
def clean_text(text):
    text = re.sub(r"[^a-zA-Z\s]", "", str(text))  # Keep only letters and spaces
    words = word_tokenize(text.lower())  # Tokenize and convert to lowercase
    words = [word for word in words if word not in stop_words]  # Remove stopwords
    return " ".join(words)

# Apply text cleaning
df["Cleaned_Reviews"] = df["Reviews"].astype(str).apply(clean_text)

# Save the updated dataset to a new CSV file
df.to_csv("cleaned_amazon_reviews.csv", index=False)
print("Cleaned data saved successfully in 'cleaned_amazon_reviews.csv'!")

# Display sample output
print(df[["Reviews", "Cleaned_Reviews"]].head())

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...


Dataset loaded successfully!
                                       Customer Name  \
0  iPhone 15 pro review - 6 months later, still g...   
1  iPhone 15 pro review - 6 months later, still g...   
2  iPhone 15 pro review - 6 months later, still g...   
3                                    Amazon Customer   
4                                    Amazon Customer   

                                 Review Title              Rating  \
0    5.0 out of 5 starsBetter than expected!!  5.0 out of 5 stars   
1    5.0 out of 5 starsBetter than expected!!  5.0 out of 5 stars   
2    5.0 out of 5 starsBetter than expected!!  5.0 out of 5 stars   
3   5.0 out of 5 starsArrived pretty flawless  5.0 out of 5 stars   
4  5.0 out of 5 starsRefurbished works great!  5.0 out of 5 stars   

                                             Reviews  
0  I never write reviews but I figured I would be...  
1  I never write reviews but I figured I would be...  
2  I never write reviews but I figured I would be...  

[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


# Question 3 (15 points)

Write a python program to **conduct syntax and structure analysis of the clean text** you just saved above. The syntax and structure analysis includes:

(1) **Parts of Speech (POS) Tagging:** Tag Parts of Speech of each word in the text, and calculate the total number of N(oun), V(erb), Adj(ective), Adv(erb), respectively.

(2) **Constituency Parsing and Dependency Parsing:** print out the constituency parsing trees and dependency parsing trees of all the sentences. Using one sentence as an example to explain your understanding about the constituency parsing tree and dependency parsing tree.

(3) **Named Entity Recognition:** Extract all the entities such as person names, organizations, locations, product names, and date from the clean texts, calculate the count of each entity.

In [5]:
import pandas as pd
import nltk
import spacy
from collections import Counter
from nltk import pos_tag, word_tokenize
from nltk.tree import Tree
from spacy import displacy
import benepar

# Load cleaned data
df = pd.read_csv("cleaned_amazon_reviews.csv")

# Load NLP models
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nlp = spacy.load("en_core_web_trf")  # Transformer model for better dependency parsing

# Load constituency parser
benepar.download('benepar_en3')
nlp.add_pipe("benepar", config={"model": "benepar_en3"})

# Display first few rows
print(df.head())


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
/usr/local/lib/python3.11/dist-packages/thinc/shims/pytorch.py:253: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling

                                       Customer Name  \
0  iPhone 15 pro review - 6 months later, still g...   
1  iPhone 15 pro review - 6 months later, still g...   
2  iPhone 15 pro review - 6 months later, still g...   
3                                    Amazon Customer   
4                                    Amazon Customer   

                                 Review Title              Rating  \
0    5.0 out of 5 starsBetter than expected!!  5.0 out of 5 stars   
1    5.0 out of 5 starsBetter than expected!!  5.0 out of 5 stars   
2    5.0 out of 5 starsBetter than expected!!  5.0 out of 5 stars   
3   5.0 out of 5 starsArrived pretty flawless  5.0 out of 5 stars   
4  5.0 out of 5 starsRefurbished works great!  5.0 out of 5 stars   

                                             Reviews  \
0  I never write reviews but I figured I would be...   
1  I never write reviews but I figured I would be...   
2  I never write reviews but I figured I would be...   
3  90% battery life, scr

# **Following Questions must answer using AI assitance**

#Question 4 (20 points).

Q4. (PART-1)
Web scraping data from the GitHub Marketplace to gather details about popular actions. Using Python, the process begins by sending HTTP requests to multiple pages of the marketplace (1000 products), handling pagination through dynamic page numbers. The key details extracted include the product name, a short description, and the URL.

 The extracted data is stored in a structured CSV format with columns for product name, description, URL, and page number. A time delay is introduced between requests to avoid server overload. ChatGPT can assist by helping with the parsing of HTML, error handling, and generating reports based on the data collected.

 The goal is to complete the scraping within a specified time limit, ensuring that the process is efficient and adheres to GitHub’s usage guidelines.

(PART -2)

1.   **Preprocess Data**: Clean the text by tokenizing, removing stopwords, and converting to lowercase.

2. Perform **Data Quality** operations.


Preprocessing:
Preprocessing involves cleaning the text by removing noise such as special characters, HTML tags, and unnecessary whitespace. It also includes tasks like tokenization, stopword removal, and lemmatization to standardize the text for analysis.

Data Quality:
Data quality checks ensure completeness, consistency, and accuracy by verifying that all required columns are filled and formatted correctly. Additionally, it involves identifying and removing duplicates, handling missing values, and ensuring the data reflects the true content accurately.


Github MarketPlace page:
https://github.com/marketplace?type=actions

In [7]:
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
from urllib.error import URLError
import csv
import requests
import time
import random

def apply_exponential_backoff(attempt):
    wait_time = min(2 ** attempt + random.uniform(0, 1), 10)
    print(f"Retrying in {wait_time:.2f} seconds...")
    time.sleep(wait_time)

session = requests.Session()
session.headers.update({'User-Agent': 'Mozilla/5.0'})

def fetch_action_links(page_num):
    page_url = f"https://github.com/marketplace?type=actions&page={page_num}"
    actions_list = []
    print(f"Scrapping Page {page_num}")

    for attempt in range(5):
        try:
            request = Request(page_url, headers={'User-Agent': 'Mozilla/5.0'})
            response = urlopen(request, timeout=10)
            soup = BeautifulSoup(response, 'html.parser')

            for action in soup.find_all("div", attrs={'data-testid': 'non-featured-item'}):
                action_anchor = action.find("a", href=True)
                action_title = action_anchor.text.strip() if action_anchor else "N/A"
                action_url = f"https://github.com{action_anchor['href']}" if action_anchor else "N/A"

                actions_list.append({
                    'Page': page_num,
                    'Action Name': action_title,
                    'URL': action_url,
                    'Description': "N/A"
                })

            print(f"Extracting {len(actions_list)} action links.")
            break
        except URLError as error:
            print(f"Error processing page {page_num}: {error}")
            apply_exponential_backoff(attempt)

    return actions_list


def fetch_action_descriptions(actions_data):
    for action in actions_data:
        action_url = action['URL']

        for attempt in range(5):
            try:
                response = session.get(action_url, timeout=10)

                if response.status_code == 200:
                    soup = BeautifulSoup(response.content, 'html.parser')
                    about_section = soup.find('div', attrs={'data-testid': 'about'})

                    if about_section:
                        description_span = about_section.find('span')
                        if description_span:
                            action['Description'] = description_span.get_text().strip()

                break
            except requests.exceptions.RequestException as error:
                print(f"Error fetching {action_url}: {error}")
                apply_exponential_backoff(attempt)

    return actions_data

def save_data_to_csv(data, filename):
    with open(filename, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=['Page', 'Action Name', 'URL', 'Description'])
        writer.writeheader()
        writer.writerows(data)

if __name__ == "__main__":
    all_actions = []

    for page in range(1, 60):
        page_actions = fetch_action_links(page)
        all_actions.extend(page_actions)

        time.sleep(5)
    enriched_actions = fetch_action_descriptions(all_actions)
    save_data_to_csv(enriched_actions, "github_actions_data.csv")
    print("Data saved to 'github_actions_data.csv'.")

Scrapping Page 1
Extracting 20 action links.
Scrapping Page 2
Extracting 20 action links.
Scrapping Page 3
Extracting 20 action links.
Scrapping Page 4
Extracting 20 action links.
Scrapping Page 5
Extracting 20 action links.
Scrapping Page 6
Extracting 20 action links.
Scrapping Page 7
Extracting 20 action links.
Scrapping Page 8
Extracting 0 action links.
Scrapping Page 9
Extracting 20 action links.
Scrapping Page 10
Extracting 0 action links.
Scrapping Page 11
Extracting 20 action links.
Scrapping Page 12
Extracting 20 action links.
Scrapping Page 13
Extracting 20 action links.
Scrapping Page 14
Extracting 20 action links.
Scrapping Page 15
Extracting 20 action links.
Scrapping Page 16
Extracting 0 action links.
Scrapping Page 17
Extracting 20 action links.
Scrapping Page 18
Extracting 20 action links.
Scrapping Page 19
Extracting 20 action links.
Scrapping Page 20
Extracting 20 action links.
Scrapping Page 21
Extracting 20 action links.
Scrapping Page 22
Extracting 20 action links.


#Question 5 (20 points)

PART 1:
Web Scrape  tweets from Twitter using the Tweepy API, specifically targeting hashtags related to subtopics (machine learning or artificial intelligence.)
The extracted data includes the tweet ID, username, and text.

Part 2:
Perform data cleaning procedures

A final data quality check ensures the completeness and consistency of the dataset. The cleaned data is then saved into a CSV file for further analysis.


**Note**

1.   Follow tutorials provided in canvas to obtain api keys. Use ChatGPT to get the code. Make sure the file is downloaded and saved.
2.   Make sure you divide GPT code as shown in tutorials, dont make multiple requestes.


In [ ]:
import tweepy
import pandas as pd

In [ ]:
# Twitter API credentials
api_key = "hJBFNJS43WU7LSVh5qP9vwP29"
api_key_secret = "nwO4qwhjJhXEgfZdZXm3fa3m36SdVJ9eh22bXaC70Z3DysgnHL"
access_token = "1892405978232213506-9X9bSMsjy7Qz3l0r3qIMrThffcAtbg"
access_token_secret = "fAxtSF7oqqz3ckv4hECPT6BqPdrPd9XxbLEzof3rCZegS"

# Authenticate with Twitter API
auth = tweepy.OAuth1UserHandler(api_key, api_key_secret, access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

In [ ]:
query = "(#MachineLearning OR #ArtificialIntelligence OR #AI OR #ML) -is:retweet"

# Use Twitter API v2
client = tweepy.Client(bearer_token="AAAAAAAAAAAAAAAAAAAAAEWozQEAAAAAtrS2ekPoriaBTYlzYDaINM5s%2FdA%3DuOUfS5GxD4pVRWv3aZ0AB6N13QvAz2EMLLtxKcZoyNaQJlQ6u1")

tweets = client.search_recent_tweets(query=query,
                                     tweet_fields=["created_at", "text", "author_id"],
                                     max_results=100)

# Store the tweets
tweet_data = []

for tweet in tweets.data:
    tweet_data.append({
        'tweet_id': tweet.id,
        'created_at': tweet.created_at,
        'user': tweet.author_id,  # API v2 does not return screen_name directly
        'text': tweet.text
    })

In [ ]:
print(len(tweet_data))  # Check how many tweets were stored
print(tweet_data[:5])   # Print first 05 tweets for verification

100
[{'tweet_id': 1892439292938420494, 'created_at': datetime.datetime(2025, 2, 20, 5, 1, 1, tzinfo=datetime.timezone.utc), 'user': 1774692567457636352, 'text': '𝗫𝗯𝗼𝘅 𝗚𝗮𝗺𝗲 𝗣𝗮𝘀𝘀: 𝗕𝗲𝘀𝘁 𝗚𝗮𝗺𝗲𝘀 𝘁𝗼 𝗣𝗹𝗮𝘆 𝗶𝗻 𝟮𝟬𝟮𝟱  Xbox Game Pass has a wide range of games to offer to player\n\nhttps://t.co/387JiuGAQJ\n\n#Xboxgames #XboxGamePass #bestXboxGamePassgames #GamingNews #AI #AINews #AnalyticsInsight #AnalyticsInsightMagazine https://t.co/ebjljckq1a'}, {'tweet_id': 1892439292305117687, 'created_at': datetime.datetime(2025, 2, 20, 5, 1, 1, tzinfo=datetime.timezone.utc), 'user': 1889043299748519937, 'text': '🏃 I’ve just started earning Bytes on @despeednet - the world’s first decentralized network for internet speed verification! 🌐\n\n⚙️ Set up your #DeSpeed Validator today and start earning Bytes.\n\n👉 Kickstart your journey here: https://t.co/R6oqoEDqkU\n\n#Depin #Extension #Web3 #AI'}, {'tweet_id': 1892439283618697529, 'created_at': datetime.datetime(2025, 2, 20, 5, 0, 59, tzinfo=datetime.timezone.utc)

In [ ]:
# Convert to DataFrame
df = pd.DataFrame(tweet_data)

# Save as CSV file
df.to_csv("tweets.csv", index=False, encoding="utf-8")

print("Tweets saved successfully to tweets.csv")

Tweets saved successfully to tweets.csv


In [ ]:
df_check = pd.read_csv("tweets.csv")
print(df_check.head())

              tweet_id                 created_at                 user  \
0  1892439292938420494  2025-02-20 05:01:01+00:00  1774692567457636352   
1  1892439292305117687  2025-02-20 05:01:01+00:00  1889043299748519937   
2  1892439283618697529  2025-02-20 05:00:59+00:00  1364079326623268866   
3  1892439273170755842  2025-02-20 05:00:57+00:00  1397046895562928129   
4  1892439272914915463  2025-02-20 05:00:57+00:00  1783262988473978880   

                                                text  
0  𝗫𝗯𝗼𝘅 𝗚𝗮𝗺𝗲 𝗣𝗮𝘀𝘀: 𝗕𝗲𝘀𝘁 𝗚𝗮𝗺𝗲𝘀 𝘁𝗼 𝗣𝗹𝗮𝘆 𝗶𝗻 𝟮𝟬𝟮𝟱  Xb...  
1  🏃 I’ve just started earning Bytes on @despeedn...  
2  Tweet:\n\n🇨🇳 China’s AI surveillance is years ...  
3  #Feet #feet #feetworshi̇p #foot #like #pretty ...  
4  👑🌊The twins are with a friend!🌊👑\n#goddessofth...  


In [ ]:
pip install nltk

In [ ]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob  # For spell checking
import emoji  # To remove emojis
import unidecode  # To normalize accented characters

# Download required NLTK resources
nltk.download("stopwords")
nltk.download("wordnet")
nltk.download("omw-1.4")

# Load the scraped tweets
df = pd.read_csv("tweets.csv")  # Update the file path if needed

# 1️⃣ Remove duplicates based on tweet_id
df.drop_duplicates(subset="tweet_id", keep="first", inplace=True)

# 2️⃣ Drop rows with missing values
df.dropna(inplace=True)

# Initialize NLP tools
lemmatizer = WordNetLemmatizer()
try:
    stop_words = set(stopwords.words("english"))
except:
    nltk.download("stopwords")
    stop_words = set(stopwords.words("english"))

# 3️⃣ Text Cleaning Function
def clean_text(text):
    text = unidecode.unidecode(text)  # Normalize accented characters
    text = re.sub(r"http\S+|www\S+", "", text)  # Remove URLs
    text = re.sub(r"@[A-Za-z0-9_]+", "", text)  # Remove mentions
    text = re.sub(r"#(\w+)", r"\1", text)  # Keep hashtags' words
    text = re.sub(r"[^\w\s]", "", text)  # Remove special characters
    text = emoji.replace_emoji(text, replace="")  # Remove emojis
    text = re.sub(r"\s+", " ", text).strip()  # Remove extra spaces
    return text.lower()  # Convert to lowercase

# 4️⃣ Remove stopwords & Lemmatization
def preprocess_text(text):
    words = text.split()
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    return " ".join(words)

# 5️⃣ Spell Checking (Optimize by applying only to short tweets)
def correct_spelling(text):
    if len(text.split()) < 5:  # Only correct short tweets to save time
        return str(TextBlob(text).correct())
    return text

# Apply all transformations in one step
df["text"] = df["text"].apply(lambda x: correct_spelling(preprocess_text(clean_text(x))))

# 6️⃣ Convert 'created_at' to datetime format
df["created_at"] = pd.to_datetime(df["created_at"], errors="coerce")

# 7️⃣ Final quality check
print("Final dataset shape:", df.shape)
print(df.head())

# Save cleaned data to a new CSV file
cleaned_filename = "final_cleaned_tweets.csv"
df.to_csv(cleaned_filename, index=False, encoding="utf-8")

print(f"🎉 Deep cleaning complete! Cleaned data saved as '{cleaned_filename}'.")


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Final dataset shape: (100, 4)
              tweet_id                created_at                 user  \
0  1892439292938420494 2025-02-20 05:01:01+00:00  1774692567457636352   
1  1892439292305117687 2025-02-20 05:01:01+00:00  1889043299748519937   
2  1892439283618697529 2025-02-20 05:00:59+00:00  1364079326623268866   
3  1892439273170755842 2025-02-20 05:00:57+00:00  1397046895562928129   
4  1892439272914915463 2025-02-20 05:00:57+00:00  1783262988473978880   

                                                text  
0  xbox game pas best game play 2025 xbox game pa...  
1  ive started earning byte world first decentral...  
2  tweet china ai surveillance year ahead facial ...  
3  foot foot feetworship foot like pretty sock to...  
4  twin friend goddessofthesea deepsea seaartai t...  
🎉 Deep cleaning complete! Cleaned data saved as 'final_cleaned_tweets.csv'.


# Mandatory Question

Provide your thoughts on the assignment. What did you find challenging, and what aspects did you enjoy? Your opinion on the provided time to complete the assignment.

# Write your response below
Fill out survey and provide your valuable feedback.

https://docs.google.com/forms/d/e/1FAIpQLSd_ObuA3iNoL7Az_C-2NOfHodfKCfDzHZtGRfIker6WyZqTtA/viewform?usp=dialog